### IMPORT LIBRARIES  

In [ ]:
from AssetAllocation.datamanger import datamanger as dm
from AssetAllocation.analytics.liability_model import liabilityModel
import pandas as pd
from ipywidgets import interact, interact_manual, interactive

# IBT or Pension or Retirement
PLAN = 'IBT'

# % Probability of Sponsor Not Contributing Service Costs
UPS_CONTRB_PCT = .05

# Target Years to be "Fully Funded"
YRS_TO_FF = 20

# Target Fully Funded Ratio
FF_RATIO = 1.05

### IMPORT CASHFLOW AND DISC RATE DATA      

In [ ]:
df_pbo_cfs = dm.get_cf_data('PBO')
df_pvfb_cfs = dm.get_cf_data('PVFB')
df_sc_cfs = df_pvfb_cfs - df_pbo_cfs
df_ftse = dm.get_ftse_data()
disc_rates = pd.read_excel(dm.TS_FP+"discount_rate_data.xlsx",sheet_name=PLAN ,usecols=[0,1],index_col=0)

### VIEW DATA

In [ ]:
data_dict = {'pbo_cashflows': df_pbo_cfs, 'sc_cashflows': df_sc_cfs,
            'pvfb_cashflows': df_pvfb_cfs,'ftse_data': df_ftse}
@interact
def display_data(data = data_dict.keys()):
    return data_dict[data]

### TRANSFORM DATA TO LIABILITY MODEL INPUTS  

In [ ]:
pbo_cashflows = df_pbo_cfs[PLAN]
disc_factors = df_pbo_cfs['Time']
sc_cashflows = df_sc_cfs[PLAN]
liab_curve = dm.generate_liab_curve(df_ftse, pbo_cashflows)
asset_mv = dm.get_plan_asset_mv(PLAN)

### CREATE LIABILITY MODEL

In [ ]:
liab_model = liabilityModel(pbo_cashflows, disc_factors, sc_cashflows, 
                                  liab_curve,disc_rates,UPS_CONTRB_PCT, asset_mv)

### COMPUTE FULFILLMENT RETURN

In [ ]:
liab_model.compute_fulfill_ret(YRS_TO_FF, FF_RATIO)
liab_model.fulfill_irr

### COMPUTE FUTURE FUNDED STATUS

In [ ]:
liab_model.fulfill_solve(liab_model.fulfill_irr, YRS_TO_FF, FF_RATIO)

### EXCESS RETURN

In [ ]:
liab_model.excess_return